In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle
import random
import tensorflow as tf

import random

from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

from tensorflow import keras

In [2]:
datacols = ['target','agegroup','fnlwgtgroup','workclass','education','maritalstatus','occupation','relationship','race','sex','cptgaingroup','cptlossgroup','hrsgroup','nativecountry']

MAX_SEQUENCE_LENGTH = len(datacols)

reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

seed = 58

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 500
projection_dim = 300
num_heads = 6

transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers

transformer_layers = 8

embed_dim = projection_dim 
embeddings_shape = (1,embed_dim)

mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

ckp_path = 'models/Model_Embedding_transformers_v3.hdf5'

In [3]:
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)

In [4]:
class DataGenerator(Sequence):
        
    def __init__(self, df_X, batch_size=32, shuffle=False):
        self.batch_size = batch_size
        self.df_X = df_X
        self.indices = self.df_X.index.tolist()
        self.vocab_size = vocab_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X, y = self.__get_data(batch)
        return X, y
    
    def n(self):
        return len(self.indices)
    
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        X1 = []
        y1 = []
        
        for i, id in enumerate(batch):
            
            column = random.randrange(MAX_SEQUENCE_LENGTH)
            
            # Data
            texts = self.df_X.iloc[self.indices[id]]
            
            #Labels
            label = texts[column]
            texts[column] = 0

            X1.append(texts)
            y1.append(label)

                            
        return np.array(X1), np.array(y1).reshape(self.batch_size,1)

In [5]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return positions + x

In [6]:
class PositionEmbeddingFixedWeights(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super(PositionEmbeddingFixedWeights, self).__init__(**kwargs)
        
        position_embedding_matrix = self.get_position_encoding(sequence_length, output_dim) 
        
        self.position_embedding_layer = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim,
            weights=[position_embedding_matrix],
            trainable=False
        )
             
    def get_position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P


    def call(self, inputs):        
        position_indices = tf.range(tf.shape(inputs)[1])
        embedded_indices = self.position_embedding_layer(position_indices)
        return inputs + embedded_indices

In [7]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [8]:
def create_classifier(learning_rate):
    
    input_text  = layers.Input(shape=(MAX_SEQUENCE_LENGTH-1,),dtype="int32",name='input_text')
    
    embeddings = tf.keras.layers.Embedding(input_dim=vocab_size, input_length=MAX_SEQUENCE_LENGTH, output_dim=embed_dim,name='embeddings')(input_text)
    
    embeddings = PositionEmbeddingFixedWeights(sequence_length=MAX_SEQUENCE_LENGTH-1,output_dim=embed_dim)(embeddings)
    
    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(embeddings)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, embeddings])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        embeddings = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(embeddings)
    representation = layers.GlobalAveragePooling1D()(representation)
    representation = layers.Dropout(0.1)(representation)
    
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.1)
    
    # Classify outputs.
    outputs = layers.Dense(num_classes,activation='softmax', name='activation')(features)
    
    # Create the Keras model.
    model = Model(inputs=input_text, outputs=outputs,name='Customer_Code_Embeddings')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),loss="sparse_categorical_crossentropy")
    
    return model

In [9]:
data = pd.read_csv('data/train_data_cat.csv')

In [10]:
# Create the lines of text
data['data'] = data[datacols].apply(' '.join, axis=1)
texts = pd.DataFrame()
texts['data'] = data['data']

In [12]:
text_lines = []
for index, row in texts.iterrows():
    text_lines.append(row['data'].split(" "))

In [13]:
text_tokens = [item for text_lines in text_lines for item in text_lines]
texts = list(np.unique(np.array(text_tokens)))
VOCAB_SIZE = len(texts)

In [14]:
print(VOCAB_SIZE)

130


In [15]:
# create the tokenizer
tokenizer = Tokenizer(filters=' ')
# fit the tokenizer on the documents
tokenizer.fit_on_texts(texts)

In [16]:
encoded_docs = tokenizer.texts_to_sequences(text_lines)

In [17]:
# integer encode the documents
vocab_size = len(tokenizer.word_index) + 1
num_classes = vocab_size

In [18]:
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [19]:
# Define the data generator
train = pd.DataFrame(padded_docs)
train_generator = DataGenerator(df_X=train, batch_size=batch_size, shuffle=True)
STEP_SIZE_TRAIN=train_generator.n()//train_generator.batch_size

In [20]:
model = create_classifier(learning_rate)
        
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.4, patience = 3, verbose = 0, min_delta = 0.001, mode = 'min')
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=0, patience=15, restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(ckp_path, monitor='loss', mode='min', verbose=0, save_best_only=True, save_weights_only=True)
        
# train the model
history = model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    callbacks=[mc,lr,es],  
                    shuffle=True)


Epoch 1/500
381/381 [==============================] - 51s 107ms/step - loss: 3.6475 - lr: 0.0010
Epoch 2/500
381/381 [==============================] - 41s 107ms/step - loss: 3.6576 - lr: 0.0010
Epoch 3/500
381/381 [==============================] - 41s 108ms/step - loss: 3.5685 - lr: 0.0010
Epoch 4/500
381/381 [==============================] - 42s 109ms/step - loss: 3.4532 - lr: 0.0010
Epoch 5/500
381/381 [==============================] - 41s 107ms/step - loss: 3.3059 - lr: 0.0010
Epoch 6/500
381/381 [==============================] - 42s 111ms/step - loss: 3.1838 - lr: 0.0010
Epoch 7/500
381/381 [==============================] - 42s 110ms/step - loss: 3.0365 - lr: 0.0010
Epoch 8/500
381/381 [==============================] - 42s 110ms/step - loss: 2.9012 - lr: 0.0010
Epoch 9/500
381/381 [==============================] - 42s 111ms/step - loss: 2.7526 - lr: 0.0010
Epoch 10/500
381/381 [==============================] - 42s 111ms/step - loss: 2.6040 - lr: 0.0010
Epoch 11/500
381/38

In [21]:
# Reload the model
model = create_classifier(learning_rate)
model.load_weights(ckp_path)

In [24]:
# Get the embedding layer weights
embedding_layer = model.get_layer('embeddings')
embeddings = embedding_layer.get_weights()[0]

In [25]:
#Save the Embeddings
np.savetxt("data/embeddings_model_w2v_v3.csv", embeddings, delimiter=",")

In [26]:
# Create the dictionaries for further processing
word_to_index = tokenizer.word_index
index_to_word = dict()

for key in word_to_index:
    index_to_word.update({word_to_index[key] : key })

word_to_index.update({'unk':0})
index_to_word.update({0:'unk'})  

In [27]:
# Save dictionaries
with open('data/word_to_index_v3.pkl', 'wb') as fp:
    pickle.dump(word_to_index, fp)
    
with open('data/index_to_word_v3.pkl', 'wb') as fp:
    pickle.dump(index_to_word, fp)    